In [1]:
from importlib import reload
import sys
import os
sys.path.insert(0, './../scripts')

In [2]:
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.figure import figaspect
import time

import spat_ori_network as network
import sim_util as su
import ricciardi as ric
import integrate as integ

def imshowbar(fig,ax,A,**kwargs):
    imsh = ax.imshow(A,**kwargs)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(imsh, cax=cax, orientation='vertical')

# Read Base Results Files

In [40]:
max_njob = 200

res_dict = {}
id_res_dict = {}

nsim = 0
for njob in range(max_njob):
    try:
        with open('./../results/results_base_'+str(njob)+'.pkl', 'rb') as handle:
            this_res_dict = pickle.load(handle)
        max_rep = max(list(this_res_dict.keys())) + 1
        for idx_rep in range(max_rep):
            res_dict[nsim] = this_res_dict[idx_rep]
            res_dict[nsim]['id'] = (njob,idx_rep)
            id_res_dict[(njob,idx_rep)] = this_res_dict[idx_rep]
            nsim += 1
    except:
        pass

print(nsim)

1311


In [41]:
prms = np.zeros((nsim,11))
inputs = np.zeros((nsim,9))
costs = np.zeros((nsim))

for idx_sim in range(nsim):
    prms[idx_sim] = np.array(list(res_dict[idx_sim]['prms'].values()))
    inputs[idx_sim] = np.concatenate(([res_dict[idx_sim]['best_eX']],[res_dict[idx_sim]['best_bX']],
                                      res_dict[idx_sim]['best_aXs']))
    costs[idx_sim] = res_dict[idx_sim]['best_cost']

In [42]:
idx_min = np.argmin(costs)

print(res_dict[idx_min]['id'])
print(res_dict[idx_min]['prms'])
print(res_dict[idx_min]['best_eX'])
print(res_dict[idx_min]['best_bX'])
print(res_dict[idx_min]['best_aXs'])
print(res_dict[idx_min]['best_means'])
print(res_dict[idx_min]['best_stds'])
print(res_dict[idx_min]['best_cost'])

(118, 5)
{'SrfE': 12.940111969332282, 'SrfI': 19.459208800703315, 'SrfF': 20, 'SoriE': 36.907244273244515, 'SoriI': 33.51951281695162, 'SoriF': 43.928905500358894, 'fEE': 0.886025501863603, 'fEI': 1.2361959979943369, 'fIE': 0.7503831441376321, 'fII': 1.0810860686494903, 'fFI': 0.8912578013409655}
0.49999999999847367
2.6102168236880745
[0.         0.25481409 0.37694752 0.53751669 0.56795482 1.52808744
 3.46419408]
[ 6.62042035  7.23251091  7.5258884   7.91159263  7.98470826 10.29104898
 15.88062207]
[ 5.68310787  6.3092617   6.60937976  7.00394586  7.07874139  9.43807213
 15.02580984]
0.9938180626068223


# Read Opto Results Files

In [87]:
max_njob = 200

res_dict = {}

nsim = 0
for njob in range(max_njob):
    try:
        with open('./../results/results_opto_'+str(njob)+'.pkl', 'rb') as handle:
            this_res_dict = pickle.load(handle)
        max_rep = max(list(this_res_dict.keys())) + 1
        for idx_rep in range(max_rep):
            res_dict[nsim] = this_res_dict[idx_rep]
            res_dict[nsim]['id'] = (njob,idx_rep)
            nsim += 1
    except:
        pass

print(nsim)

813


In [88]:
prms = np.zeros((nsim,11))
inputs = np.zeros((nsim,2))
costs = np.zeros((nsim))

for idx_sim in range(nsim):
    prms[idx_sim] = np.array(list(res_dict[idx_sim]['prms'].values()))
    inputs[idx_sim] = np.concatenate(([res_dict[idx_sim]['best_L']],[res_dict[idx_sim]['best_CVL']]))
    costs[idx_sim] = res_dict[idx_sim]['best_cost'] + id_res_dict[res_dict[idx_sim]['id']]['best_cost']
    costs[idx_sim] *= 1e8 if res_dict[idx_min]['best_diff_means'][-1] > res_dict[idx_min]['best_diff_means'][0] \
        else 1

In [89]:
idx_min = np.argmin(costs)

print(res_dict[idx_min]['id'])
print(res_dict[idx_min]['prms'])
print(id_res_dict[res_dict[idx_sim]['id']]['best_eX'])
print(id_res_dict[res_dict[idx_sim]['id']]['best_bX'])
print(id_res_dict[res_dict[idx_sim]['id']]['best_aXs'])
print(res_dict[idx_min]['best_L'])
print(res_dict[idx_min]['best_CVL'])
print(id_res_dict[res_dict[idx_sim]['id']]['best_means'])
print(id_res_dict[res_dict[idx_sim]['id']]['best_stds'])
print(res_dict[idx_min]['best_opto_means'])
print(res_dict[idx_min]['best_opto_stds'])
print(res_dict[idx_min]['best_diff_means'])
print(res_dict[idx_min]['best_diff_stds'])
print(res_dict[idx_min]['best_norm_covs'])
# print(res_dict[idx_min]['best_cost'] + id_res_dict[res_dict[idx_sim]['id']]['best_cost'])
print(costs[idx_min])

(84, 0)
{'SrfE': 15.7538486994389, 'SrfI': 7.046622992595769, 'SrfF': 20, 'SoriE': 24.953877993731425, 'SoriI': 38.12048547450283, 'SoriF': 36.75758272019653, 'fEE': 0.9609543331794965, 'fEI': 1.086519954916826, 'fIE': 0.9911424325463908, 'fII': 1.2252079589178013, 'fFI': 1.2014852358170565}
0.49999999999262096
2.5595960598498584
[0.         0.38566975 0.57793687 0.8321163  0.87109781 2.25731141
 5.00923031]
2.23158283227162
1.778279414270379
[ 6.80984158  7.32339518  7.57941597  7.91787851  7.96978586  9.89367612
 14.52970854]
[ 5.69153654  6.30835027  6.61584911  7.02236625  7.08471059  9.4959045
 16.18880405]
[10.33878627 10.96994928 11.27091742 11.67574667 11.74358565 14.32587743
 20.07854425]
[10.06069272 10.4790371  10.69050148 10.98620252 11.03695812 13.17753959
 18.73136926]
[3.95992366 4.07144156 4.1213966  4.18550426 4.19593014 4.5316352
 4.98703814]
[8.2034326  8.29458033 8.33565555 8.38860427 8.39722868 8.67952918
 9.07073767]
[0.04174232 0.04753108 0.05052136 0.05471933 0.

# Read Comb Results Files

In [90]:
max_njob = 200

res_dict = {}

nsim = 0
for njob in range(max_njob):
    try:
        with open('./../results/results_comb_'+str(njob)+'.pkl', 'rb') as handle:
            this_res_dict = pickle.load(handle)
        max_rep = max(list(this_res_dict.keys())) + 1
        for idx_rep in range(max_rep):
            res_dict[nsim] = this_res_dict[idx_rep]
            res_dict[nsim]['id'] = (njob,idx_rep)
            nsim += 1
    except:
        pass

print(nsim)

507


In [91]:
prms = np.zeros((nsim,13))
inputs = np.zeros((nsim,9))
costs = np.zeros((nsim))

for idx_sim in range(nsim):
    prms[idx_sim] = np.array(list(res_dict[idx_sim]['prms'].values()))
    inputs[idx_sim] = np.concatenate(([res_dict[idx_sim]['best_eX']],[res_dict[idx_sim]['best_bX']],
                                      res_dict[idx_sim]['best_aXs']))
    costs[idx_sim] = res_dict[idx_sim]['best_cost']
    costs[idx_sim] *= 1e8 if res_dict[idx_min]['best_diff_means'][-1] > res_dict[idx_min]['best_diff_means'][0] \
        else 1

In [92]:
idx_min = np.argmin(costs)

print(res_dict[idx_min]['id'])
print(res_dict[idx_min]['prms'])
print(res_dict[idx_min]['best_eX'])
print(res_dict[idx_min]['best_bX'])
print(res_dict[idx_min]['best_aXs'])
print(res_dict[idx_min]['best_base_means'])
print(res_dict[idx_min]['best_base_stds'])
print(res_dict[idx_min]['best_opto_means'])
print(res_dict[idx_min]['best_opto_stds'])
print(res_dict[idx_min]['best_diff_means'])
print(res_dict[idx_min]['best_diff_stds'])
print(res_dict[idx_min]['best_norm_covs'])
# print(res_dict[idx_min]['best_cost'])
print(costs[idx_min])

(45, 2)
{'SrfE': 10.550152081539142, 'SrfI': 16.07865249050877, 'SrfF': 30, 'SoriE': 28.184809695055524, 'SoriI': 30.21989163195262, 'SoriF': 39.15714170801061, 'fEE': 1.0506607239469055, 'fEI': 0.7467693106136719, 'fIE': 0.9187698061837739, 'fII': 0.8254365414159728, 'fFI': 1.3917451440450372, 'L': 2.8037678888031996, 'CVL': 1.4346284857676583}
0.5399859632931963
2.372266912228103
[0.         0.1593714  0.25965879 0.43302212 0.51366734 1.10942887
 2.91251707]
[ 6.52127529  6.94659229  7.21423085  7.67688835  7.89210752  9.48202584
 14.29395615]
[ 5.55592329  6.02652041  6.32265233  6.8345653   7.07269714  8.83188145
 14.15609979]
[11.06714501 11.57496653 11.8945226  12.44692807 12.70389623 14.60223265
 20.34759858]
[10.52801663 10.93750511 11.1951833  11.64062263 11.84783225 13.37858051
 18.01143102]
[4.54586972 4.62837425 4.68029175 4.77003972 4.81178871 5.12020681
 6.05364243]
[7.8861239  7.93682822 7.96873485 8.02389073 8.04954817 8.23909086
 8.81274687]
[0.10188507 0.11088002 0.11